In [1]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import tensorflow as tf
import pickle
import sys
import yaml

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten, TimeDistributed, Bidirectional
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
from keras.utils import plot_model
from keras import optimizers
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import load_model
from sklearn import preprocessing
from copy import deepcopy

import datetime
import math
import hashlib
import time
import os
from datagenerator import DataGenerator

# add
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used
# add
config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 
# # sess = tf.Session(config=config) 
sess = tf.compat.v1.Session(config=config) 

def loadConfig():
#     with open('/home/wesleyjtann/ddos_research/test/light_config_b.yaml', "r") as ymlfile:
    with open('./light_config.yaml', "r") as ymlfile: #GE, hulk, slowhttp, slowloris, heartbleed
        cfg = yaml.load(ymlfile)
    return cfg

config = loadConfig()
# artefact = 'artefact' #'artefact_b' #

Using TensorFlow backend.
/home/wesleyjtann/miniconda3/envs/ddos/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wesleyjtann/miniconda3/envs/ddos/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wesleyjtann/miniconda3/envs/ddos/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/we

In [2]:
df_test = pd.read_csv(config['metadata']['uniqueID'] + '/artefact/TEST.csv')
df_test

,Source IP,Dest IP,Protocols,Info,Input,Histo
0,0,0,eth:ethertype:ipv6:udp:dhcpv6,Solicit XID: 0x41d5a3 CID: 0001000120d5961f001...,122<SEP>122<SEP>122<SEP>122<SEP>122<SEP>122<SE...,Solicit XID: 0x41d5a3 CID: 0001000120d5961f001...
1,0,0,eth:ethertype:arp,Who has 192.168.10.1? Tell 192.168.10.19,169<SEP>169<SEP>122<SEP>122<SEP>122<SEP>122<SE...,Who has 192.168.10.1? Tell 192.168.10.19<JOIN>...
2,0,0,eth:ethertype:ipv6:udp:dhcpv6,Solicit XID: 0xdf7ba9 CID: 0001000120d46426b8a...,122<SEP>122<SEP>122<SEP>122<SEP>122<SEP>122<SE...,Solicit XID: 0xdf7ba9 CID: 0001000120d46426b8a...
3,0,0,eth:ethertype:ipv6:udp:mdns,"Standard query 0x0000 PTR _ipps._tcp.local, ""Q...",158<SEP>158<SEP>158<SEP>158<SEP>158<SEP>158<SE...,"Standard query 0x0000 PTR _ipps._tcp.local, ""Q..."
4,0,0,eth:ethertype:ipv6:udp:dhcpv6,Solicit XID: 0x41d5a3 CID: 0001000120d5961f001...,122<SEP>122<SEP>122<SEP>122<SEP>122<SEP>122<SE...,Solicit XID: 0x41d5a3 CID: 0001000120d5961f001...
...,...,...,...,...,...,...
19213,52.84.143.97,192.168.10.15,eth:ethertype:ip:tcp,[TCP Keep-Alive ACK] 80 → 59397 [ACK] Seq=980 ...,232<SEP>232,[TCP Keep-Alive ACK] 80 → 59397 [ACK] Seq=980 ...
19214,69.4.95.11,192.168.10.15,eth:ethertype:ip:tcp,[TCP Keep-Alive ACK] 80 → 59347 [ACK] Seq=4307...,19<SEP>210,[TCP Keep-Alive ACK] 80 → 59347 [ACK] Seq=4307...
19215,72.21.91.29,192.168.10.15,eth:ethertype:ip:tcp,[TCP Keep-Alive ACK] 80 → 59262 [ACK] Seq=2389...,103<SEP>192,[TCP Keep-Alive ACK] 80 → 59262 [ACK] Seq=2389...
19216,72.21.91.29,192.168.10.9,eth:ethertype:ip:tcp,[TCP Keep-Alive ACK] 80 → 9651 [ACK] Seq=797 A...,210<SEP>210,[TCP Keep-Alive ACK] 80 → 9651 [ACK] Seq=797 A...


In [3]:
df_norm = pd.read_csv(config['metadata']['uniqueID'] + '/artefact/N2.csv')

print(len(df_norm))
df_norm.head()

25506


,Source IP,Dest IP,Protocols,Info,Input,Histo
0,0,0,eth:ethertype:ipv6:udp:dhcpv6,Solicit XID: 0xfbb93d CID: 0001000120d46426b8a...,143<SEP>143<SEP>143<SEP>143<SEP>143<SEP>143<SE...,Solicit XID: 0xfbb93d CID: 0001000120d46426b8a...
1,0,0,eth:ethertype:arp,192.168.10.17 is at 00:23:ae:9b:95:67,186<SEP>143<SEP>143<SEP>143<SEP>143<SEP>143<SE...,192.168.10.17 is at 00:23:ae:9b:95:67<JOIN>eth...
2,0,0,eth:ethertype:arp,192.168.10.1 is at 00:c1:b1:14:eb:31,186<SEP>143<SEP>143<SEP>143<SEP>143<SEP>143<SE...,192.168.10.1 is at 00:c1:b1:14:eb:31<JOIN>eth:...
3,0,0,eth:ethertype:ipv6:udp:dhcpv6,Solicit XID: 0xd512a2 CID: 0001000120d5ad0eb8a...,143<SEP>143<SEP>143<SEP>143<SEP>143<SEP>143<SE...,Solicit XID: 0xd512a2 CID: 0001000120d5ad0eb8a...
4,0,0,eth:ethertype:ipv6:udp:dhcpv6,Solicit XID: 0xfbb93d CID: 0001000120d46426b8a...,143<SEP>143<SEP>143<SEP>143<SEP>143<SEP>143<SE...,Solicit XID: 0xfbb93d CID: 0001000120d46426b8a...


In [4]:
df_norm = df_norm.sample(frac=1)

# df_normN1 = df_norm[:35000] #[:35263]
df_normN2 = df_norm[-2506:] #[35264:]

## Save new N1
# df_normN1.to_csv('../1_testmimic_caida07/ids_test2/artefact/N1_new.csv',index=False)

In [5]:
len(df_normN2)

2506

In [6]:
## True normals

# df_norm = df_test[(df_test['Attack']==0)]
# df_norm = df_test

df_normN2['Attack']=0

print("length df_norm: ",  len(df_normN2)) 

length df_norm:  2506


/home/wesleyjtann/miniconda3/envs/ddos/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
df_normN2.to_csv('../1_testmimic_cicFriday/ids_test2/artefact/TESTnorm.csv', index=False)

In [ ]:
3262/.2